In [24]:
import json
import pickle
import re

In [18]:
cand_pat = re.compile(r"(?<=\<).+?(?=\>)")
cand_str_pat = re.compile(r"(?<=The referent entity candidates are:).*?(?=\. What is the correct referent entity for the entity mention)")

In [19]:
# Load input data
with open("./turl_test_2k_prompts_50.jsonl", "r", encoding="utf-8") as f:
    prompts = [json.loads(line) for line in f]

In [25]:
# Modify prompts so that candidates are unique
for pid, prompt in enumerate(prompts):
    cand_str = cand_str_pat.findall(prompt["question"])[0]
    candidates = cand_pat.findall(cand_str)
    unique_candidates = list(set(candidates))
    try:
        prompt["question"] = re.sub(
            cand_str_pat,
            ",".join(["<" + cand + ">" for cand in unique_candidates]),
            prompt["question"],
        )
    except Exception as e:
        print("Error in prompt:", pid)
        print(e)
        continue
with open("./turl_test_2k_prompts_50_unique.jsonl", "w", encoding="utf-8") as f:
    for prompt in prompts:
        f.write(json.dumps(prompt) + "\n")

Error in prompt: 38
bad escape \p at position 5174
Error in prompt: 544
bad escape \e at position 12958
Error in prompt: 1302
bad escape \m at position 7844


In [26]:
# Load input data
with open("./turl_test_2k_prompts_50_unique.jsonl", "r", encoding="utf-8") as f:
    prompts_unique = [json.loads(line) for line in f]

In [35]:
saved_pids = []
min_candidates = 10
for pid, p in enumerate(prompts_unique):
    candidates = cand_pat.findall(p["question"])
    if len(candidates) < min_candidates:
        continue
    if len(set(candidates)) != len(candidates):
        print(f"Duplicate candidates in prompt {pid}")
        unique_candidates = list(set(candidates))
        continue
    saved_pids.append(pid)
with open(f"./selected_pids.{len(saved_pids)}.pickle", "wb") as f:
    pickle.dump(set(saved_pids), f)

Duplicate candidates in prompt 38
Duplicate candidates in prompt 1302
